# Feb 2, 2025: high resolution Allen parcellation CCFv2

In [1]:
import numpy as np
import SimpleITK as sitk
import nibabel as nib
import ants

In [2]:
BASE_path = f'/home/govindas/new_mouse_dataset'
ATLAS_path = f'{BASE_path}/allen_atlas_ccfv2/atlasVolume'

In [3]:
mhd_file = f'{ATLAS_path}/atlasVolume.mhd'

# load the metaimage 
image = sitk.ReadImage(mhd_file)
data = sitk.GetArrayFromImage(image)

# get affine information
spacing = image.GetSpacing()
origin = image.GetOrigin()
direction = image.GetDirection()

# create affine transformation for nifti
affine = np.eye(4)
affine[:3, :3] *= np.array(spacing)[:, None] / 1000 # in mm, not um
affine[:3, 3] = origin

# convert to nifti
nii_img = nib.Nifti1Image(data, affine)
nii_file = f'{ATLAS_path}/atlasVolume_25um.nii.gz'
nib.save(nii_img, nii_file)

In [4]:
# linearly transform the template to match the parcellation (derived from CCFv2, that Francesca shared)
temp_img = ants.image_read(nii_file)
atlas_img = ants.image_read(f'{BASE_path}/parcellation/allen_space/Atlas2014_LPS.nii.gz')

In [5]:
atlas_img, temp_img

(ANTsImage (RAI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (456, 320, 528)
 	 Spacing    : (0.025, 0.025, 0.025)
 	 Origin     : (5.78, 7.83, -6.93)
 	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.],
 ANTsImage (LPI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (456, 320, 528)
 	 Spacing    : (0.025, 0.025, 0.025)
 	 Origin     : (0.0, 0.0, 0.0)
 	 Direction  : [-1.  0.  0.  0. -1.  0.  0.  0.  1.])

In [6]:
tx = ants.registration(
    fixed=atlas_img,
    moving=temp_img,
    type_of_transform='QuickRigid',
    random_seed=100,
)

In [7]:
temp_tx_img = ants.apply_transforms(
    fixed=atlas_img,
    moving=temp_img,
    transformlist=tx['fwdtransforms']
)

In [8]:
atlas_img, temp_tx_img

(ANTsImage (RAI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (456, 320, 528)
 	 Spacing    : (0.025, 0.025, 0.025)
 	 Origin     : (5.78, 7.83, -6.93)
 	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.],
 ANTsImage (RAI)
 	 Pixel Type : float (float32)
 	 Components : 1
 	 Dimensions : (456, 320, 528)
 	 Spacing    : (0.025, 0.025, 0.025)
 	 Origin     : (5.78, 7.83, -6.93)
 	 Direction  : [1. 0. 0. 0. 1. 0. 0. 0. 1.])

In [10]:
nii_tx_file = f'{ATLAS_path}/atlasVolume_tx_25um.nii.gz'
temp_tx_vol = temp_tx_img.numpy()
temp_tx_vol = temp_tx_vol[::-1, :, :]
temp_tx_img = temp_tx_img.new_image_like(temp_tx_vol)
temp_tx_img.to_file(nii_tx_file)